In [ ]:
# Import Library
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo
from pymongo import MongoClient

from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

In [2]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [3]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
)

In [4]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq']

In [5]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
)

In [6]:
# Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Resmi Allo Bank Indonesia.
    Tugas Anda adalah menjawab pertanyaan berdasarkan data FAQ resmi dari website Allo Bank.

    Tanggung jawab utama:
    Memberikan jawaban akurat dan informatif sesuai dengan kategori FAQ: umum, akun, produk, layanan, proses pengaduan nasabah, dan registrasi.
    Menyampaikan informasi dengan bahasa profesional, jelas, dan mudah dipahami.
    Memberikan panduan teknis (jika diperlukan) dalam format numerik atau langkah-langkah berurutan.
    Menanggapi keluhan atau kendala nasabah dengan solusi yang relevan dari FAQ.

    Aturan jawaban:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi.
    2. Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan bahwa Anda tidak memiliki informasi tersebut.
    3. Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    4. Jangan mencantumkan link eksternal, tautan promosi, atau informasi tambahan di luar konteks.
    5. Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [ ]:
# Model Configuration dengan GPT-3.5 turbo
llm_gpt35 = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=OPENAI_KEY,
    temperature=0,
)

In [ ]:
# Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_gpt35,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
)

In [ ]:
faq_df = pd.read_csv('Dataset/faq_allobank_clean.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

# Pertanyaan di dalam konteks Allo Bank

In [14]:
# Test Cases 1
questions = [
    "Apa itu Allo Bank?",
    "Mengapa saya tidak bisa menggunakan Allo Pay saya?",
    "Mengapa saya tidak bisa log in di aplikasi Allo Bank?",
    "Apakah menyimpan dana di Allo Bank dipastikan aman?",
    "Mengapa aplikasi Allo Bank saya berjalan sangat lambat?",
    "Bagaimana jika saldo atau limit saya tidak cukup?",
    "Keuntungan apa yang saya dapatkan sebagai member Allo Bank?",
    "Mengapa saya tidak bisa mendaftarkan nomor ponsel/email di aplikasi Allo Bank?",
    "Biaya apa saja yang dikenakan ketika melakukan registrasi?",
    "Bagaimana cara membuat rekening Allo Bank?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Apa itu Allo Bank?
Allo Bank adalah sebuah layanan perbankan berbasis digital yang dikembangkan oleh Allo Bank Indonesia. Aplikasi Allo memungkinkan pengguna untuk menikmati berbagai fitur perbankan dalam satu aplikasi yang mudah diakses. Allo Bank didirikan di Indonesia pada tahun 2021 dan bertujuan untuk memudahkan nasabah dalam melakukan kegiatan finansial dan non-finansial melalui platform digital.

ACTUAL ANSWER: Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan "All in One" dan mempunyai arti sebagai "one for all, and all for one" alias semua untuk satu dan satu untuk semua.

Pertanyaan: Mengapa saya tidak bisa menggunakan Allo Pay saya?
Anda mungkin tidak dapat menggunakan Allo Pay Anda karena beberapa alasan berikut:
1. Akun Anda bukan merupakan akun Allo Pay+ atau Allo Prime.
2. Akun An

# Pertanyaan di luar Konteks Allo Bank

In [15]:
# Test Cases 2
questions = [
    "Siapa presiden dan wakil presiden pertama Indonesia?",
    "Kapan Indonesia merdeka?",
    "Apakah alien itu benar-benar ada?",
    "Bagaimana cara belajar coding untuk pemula?",
    "Siapa penemu bom atom?",
    "Siapa itu Oppenheimer?",
    "Siapa itu Christoper Nolan?",
    "Apa itu teori relativitas?",
    "Apakah black hole itu nyata?",
    "Mengapa bumi itu bulat?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(result['result'])
    print(f"\nACTUAL ANSWER: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Siapa presiden dan wakil presiden pertama Indonesia?
Maaf, pertanyaan tersebut tidak relevan dengan informasi yang tersedia dalam FAQ resmi Allo Bank Indonesia. Jika Anda memiliki pertanyaan lain terkait dengan produk, layanan, atau prosedur di Allo Bank, silakan berikan pertanyaan yang lebih sesuai. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Kapan Indonesia merdeka?
Maaf, pertanyaan tersebut tidak relevan dengan informasi yang tersedia dalam FAQ resmi Allo Bank Indonesia. Jika Anda memiliki pertanyaan lain terkait dengan produk, layanan, atau prosedur di Allo Bank, silakan sampaikan agar saya dapat membantu dengan informasi yang sesuai. Terima kasih.

ACTUAL ANSWER: Tidak ada jawaban.

Pertanyaan: Apakah alien itu benar-benar ada?
Maaf, pertanyaan tersebut tidak relevan dengan FAQ resmi Allo Bank Indonesia. Jadi, saya tidak memiliki informasi terkait hal tersebut. Apakah ada pertanyaan lain yang dapat saya bantu?

ACTUAL ANSWER: Tidak ada jawaban.

Pert

# Kesimpulan

Dengan menggunakan model LLM <b>gpt-3.5-turbo</b>, dapat diketahui bahwa model ini mampu menjawab pertanyaan yang relevan dengan konteks project, yaitu FAQ Allo Bank, secara langsung, ringkas, dan tepat sasaran berdasarkan data yang tersedia dalam database. Selain itu, model juga menunjukkan kemampuan untuk memberikan jawaban atas pertanyaan yang masih berkaitan dengan konteks project, meskipun informasi tersebut tidak secara eksplisit terdapat dalam database.

Model ini juga mampu memberikan respons dengan sopan dan profesional ketika dihadapkan pada pertanyaan di luar konteks Allo Bank. Dalam kasus tersebut, model secara tepat menolak untuk menjawab dan menetapkan batasan konteks, sesuai dengan tujuan prompting yang telah ditentukan.